In [ ]:
import os
import pandas
import numpy
import matplotlib.pyplot as plt
import seaborn
from scipy import stats


In [ ]:
current_directory = "/Users/mlabhishek/Documents/Assignment for DS Candidates"

In [ ]:
data = pandas.read_csv(r"loan_data.csv")

In [ ]:
data.head(10)

In [ ]:
data.info()

In [ ]:
NA_col = data.isnull().sum()
NA_col = NA_col[NA_col.values >(0.2*len(data))]
plt.figure(figsize=(20,4))
NA_col.plot(kind='bar')
plt.title('Columns List & NA value counts more than 30%')
plt.show()

In [ ]:
NA_col[NA_col.values >(0.2*len(data))]

In [ ]:
data.isnull().sum()/len(data)*100

In [ ]:
data.isnull().sum()

In [ ]:
## Data Cleaning function for handling nulls and remove Nulls ##

In [ ]:
def removeNulls(dataframe, axis =1, percent=0.3):
    df = dataframe.copy()
    ishape = df.shape
    if axis == 0:
        rownames = df.transpose().isnull().sum()
        rownames = list(rownames[rownames.values > percent*len(df)].index)
        df.drop(df.index[rownames],inplace=True) 
        print("nNumber of Rows droppedt: ",len(rownames))
    else:
        colnames = (df.isnull().sum()/len(df))
        colnames = list(colnames[colnames.values>=percent].index)
        df.drop(labels = colnames,axis =1,inplace=True)        
        print("Number of Columns droppedt: ",len(colnames))
    print("nOld dataset rows,columns",ishape,"nNew dataset rows,columns",df.shape)
    return df

In [ ]:
# Remove columns where NA values are more than or equal to 30%
data = removeNulls(data, axis =1,percent = 0.3)

In [ ]:
# Remove any rows with NA values greater than or equal to 30%.
data = removeNulls(data, axis =1,percent = 0.3)

In [ ]:
# Remove all columns with only one unique value.
unique = data.nunique()
unique = unique[unique.values == 1]

In [ ]:
data.drop(labels = list(unique.index), axis =1, inplace=True)
print("So now we are left with",data.shape ,"rows & columns.")

In [ ]:
#nEmployment Term: Replace the value of ‘n/a’ with self-employed.’
print(data.emp_length.unique())
data.emp_length.fillna('0',inplace=True)
data.emp_length.replace(['n/a'],'Self-Employed',inplace=True)
print(data.emp_length.unique())

In [ ]:
print(data.emp_length.unique())

In [ ]:
print(data.zip_code.unique())

In [ ]:
# Remove any columns that aren’t relevant.

In [ ]:
not_required_columns = ["id","member_id","url","zip_code"]
data.drop(labels = not_required_columns, axis =1, inplace=True)
print("So now we are left with",data.shape ,"rows & columns.")

In [ ]:
plt.figure(figsize=(20,30))
seaborn.heatmap(data.corr(),annot=True)
plt.title('Correlation Matrix (for Loan Status)')

In [ ]:
# Convert all continuous variables to numeric values.
numeric_columns = ['loan_amnt','funded_amnt','funded_amnt_inv','installment','int_rate','annual_inc','dti']
data[numeric_columns] = data[numeric_columns].apply(pandas.to_numeric)

In [ ]:
data[numeric_columns] = data[numeric_columns].apply(pandas.to_numeric)

In [ ]:
# Loan purpose: Remove records with values less than 0.75%.
(data.purpose.value_counts()*100)/len(data)

In [ ]:
data.purpose.value_counts()

In [ ]:
del_loan_purpose = (data.purpose.value_counts()*100)/len(data)
del_loan_purpose = del_loan_purpose[(del_loan_purpose < 0.75) | (del_loan_purpose.index == 'other')]
data.drop(labels = data[data.purpose.isin(del_loan_purpose.index)].index, inplace=True)
print("So now we are left with",data.shape ,"rows & columns.")
print(data.purpose.unique())

In [ ]:
# Loan Status: Remove all records with a value of less than 1.5%.
(data.loan_status.value_counts()*100)/len(data)

In [ ]:
del_loan_status = (data.loan_status.value_counts()*100)/len(data)
del_loan_status = del_loan_status[(del_loan_status < 1.5)]
data.drop(labels = data[data.loan_status.isin(del_loan_status.index)].index, inplace=True)
print("So now we are left with",data.shape ,"rows & columns.")
print(data.loan_status.unique())

In [ ]:
(data.loan_status.value_counts()*100)/len(data)

In [ ]:
# Purpose of Loan / Loan Amount for loan status
plt.figure(figsize=(16,12))
data['loan_amnt'] = data['loan_amnt'].astype('float')
seaborn.boxplot(data =data, x='purpose', y='loan_amnt', hue ='loan_status')
plt.title('Purpose of Loan vs Loan Amount')
plt.show()

In [ ]:
# univariate plots
def univariate(df,col,vartype,hue =None):    
    '''
    Univariate function will plot parameter values in graphs.
    df      : dataframe name
    col     : Column name
    vartype : variable type : continuous or categorical
                Continuous(0)   : Distribution, Violin & Boxplot will be plotted.
                Categorical(1) : Countplot will be plotted.
    hue     : Only applicable in categorical analysis.
    '''
    seaborn.set(style="darkgrid")
    if vartype == 0:
        fig, ax=plt.subplots(nrows =1,ncols=3,figsize=(20,8))
        ax[0].set_title("Distribution Plot")
        seaborn.distplot(df[col],ax=ax[0])
        ax[1].set_title("Violin Plot")
        seaborn.violinplot(data =df, x=col,ax=ax[1], inner="quartile")
        ax[2].set_title("Box Plot")
        seaborn.boxplot(data =df, x=col,ax=ax[2],orient='v')
    if vartype == 1:
        temp = pandas.Series(data = hue)
        fig, ax = plt.subplots()
        width = len(df[col].unique()) + 6 + 4*len(temp.unique())
        fig.set_size_inches(width , 7)
        ax = seaborn.countplot(data = df, x= col, order=df[col].value_counts().index,hue = hue) 
        if len(temp.unique()) > 0:
            for p in ax.patches:
                ax.annotate('{:1.1f}%'.format((p.get_height()*100)/float(len(data))), (p.get_x()+0.05, p.get_height()+20))  
        else:
            for p in ax.patches:
                ax.annotate(p.get_height(), (p.get_x()+0.32, p.get_height()+20)) 
        del temp
    else:
        exit
    plt.show()

In [ ]:
# loan amount
univariate(df=data,col='loan_amnt',vartype=0)

In [ ]:
# Interest Rate
data['int_rate'] = data['int_rate'].replace("%","", regex=True).astype(float)
univariate(df=data,col='int_rate',vartype=0)

In [ ]:
# Annual Income
data["annual_inc"].describe()

In [ ]:
# Remove Outliers (values from 99 to 100%)
q = data["annual_inc"].apply(lambda x: float(x)).quantile(0.995)
data = data[data["annual_inc"].apply(lambda x: float(x)) < q]
data["annual_inc"].describe()

In [ ]:
data['annual_inc'] = data['annual_inc'].apply(lambda x: float(x))
univariate(df=data,col='annual_inc',vartype=0)

In [ ]:
# Loan Status
univariate(df=data,col='loan_status',vartype=1)

In [ ]:
# Home Ownership Wise Loan
data.purpose.unique()

In [ ]:
data.home_ownership.unique()

In [ ]:
#  Remove rows where home_ownership’==’OTHER’, ‘NONE’, ‘ANY’
rem = ['OTHER', 'NONE', 'ANY']
data.drop(data[data['home_ownership'].isin(rem)].index,inplace=True)
data.home_ownership.unique()

In [ ]:
univariate(df=data,col='home_ownership',vartype=1,hue='loan_status')

In [ ]:
data.columns.value_counts()

In [ ]:
data['loan_status'].nunique()

In [ ]:
data.describe()